In [11]:
# standard imports
from pylabrobot.liquid_handling.backends.backend import LiquidHandlerBackend
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend
from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.visualizer.visualizer import Visualizer

# resources for deck setup
from pylabrobot.resources import (
    Deck,
    set_tip_tracking,
    set_volume_tracking,
    set_cross_contamination_tracking,
    corning_96_wellplate_360ul_flat,
    opentrons_96_tiprack_1000ul,
    agilent_1_reservoir_290ml
)

In [12]:
set_tip_tracking(enabled = True)
set_volume_tracking(enabled = True)
set_cross_contamination_tracking(enabled = True)

In [13]:
async def visualize_deck(deck: Deck,
                         backend: LiquidHandlerBackend):
    try:
        lh = LiquidHandler(backend=backend, deck=deck)
        vis = Visualizer(resource = lh)
        await lh.setup()
        await vis.setup()
        return lh
    except Exception as e:
        print(f"Error! Got excpetion: {e}")

In [18]:
# create a list of all tip positions in order
tip_positions = []
for row in 'ABCDEFGH':
    for col in range(1, 13):
        tip_positions.append(f'{row}{col}')

# track which tip we're on
current_tip_index = 0
current_rack_index = 0

async def get_next_tip_spot(lh):
    global current_tip_index, current_rack_index
    
    tip_racks = [lh.deck.get_resource("tip_rack_0"),
                 lh.deck.get_resource("tip_rack_1"),
                 lh.deck.get_resource("tip_rack_2")]
    
    current_rack = tip_racks[current_rack_index]
    tip_position = tip_positions[current_tip_index]
    tip_spot = current_rack[tip_position]  
    
    current_tip_index += 1
    if current_tip_index >= 96:
        current_tip_index = 0
        current_rack_index += 1
    
    return tip_spot

In [ ]:
def get_digit_wells(digit_value, col_start):
    """
    Get wells for a digit spanning 3 columns (col_start to col_start+2)
    Rows A-E form the 7-segment display
    """
    col_left = col_start
    col_mid = col_start + 1
    col_right = col_start + 2
    
    segments = {
        0: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_left}', f'C{col_left}', f'D{col_left}',  # left side
            f'B{col_right}', f'C{col_right}', f'D{col_right}',  # right side
            f'E{col_left}', f'E{col_mid}', f'E{col_right}'],  # bottom
        
        1: [f'A{col_right}', f'B{col_right}', f'C{col_right}', 
            f'D{col_right}', f'E{col_right}'],  # right side only
        
        2: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_right}', f'C{col_right}',  # top-right
            f'C{col_mid}',  # middle
            f'D{col_left}', f'E{col_left}',  # bottom-left
            f'E{col_left}', f'E{col_mid}', f'E{col_right}'],  # bottom
        
        3: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_right}', f'C{col_right}', f'D{col_right}', f'E{col_right}',  # right side
            f'C{col_mid}',  # middle
            f'E{col_left}', f'E{col_mid}'],  # bottom
        
        4: [f'A{col_left}', f'B{col_left}', f'C{col_left}',  # top-left
            f'C{col_mid}',  # middle
            f'A{col_right}', f'B{col_right}', f'C{col_right}', f'D{col_right}', f'E{col_right}'],  # right side
        
        5: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_left}', f'C{col_left}',  # top-left
            f'C{col_mid}',  # middle
            f'D{col_right}', f'E{col_right}',  # bottom-right
            f'E{col_left}', f'E{col_mid}'],  # bottom
        
        6: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_left}', f'C{col_left}', f'D{col_left}', f'E{col_left}',  # left side
            f'C{col_mid}',  # middle
            f'D{col_right}', f'E{col_right}',  # bottom-right
            f'E{col_mid}'],  # bottom
        
        7: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_right}', f'C{col_right}', f'D{col_right}', f'E{col_right}'],  # right side
        
        8: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_left}', f'B{col_right}',  # row B sides
            f'C{col_left}', f'C{col_mid}', f'C{col_right}',  # row C (middle segment)
            f'D{col_left}', f'D{col_right}',  # row D sides
            f'E{col_left}', f'E{col_mid}', f'E{col_right}'],  # bottom
        
        9: [f'A{col_left}', f'A{col_mid}', f'A{col_right}',  # top
            f'B{col_left}', f'B{col_right}',  # row B both sides
            f'C{col_left}', f'C{col_mid}', f'C{col_right}',  # middle
            f'D{col_right}', f'E{col_right}'],  # right side bottom
    }
    
    return segments[digit_value]

async def display_number(lh, number: int):
    if number < 0 or number > 9999:
        raise ValueError("Number must be between 0 and 9999")
    
    dino_plate = lh.deck.get_resource("dino_plate")
    trough = lh.deck.get_resource("trough")
    
    digits_str = str(number).zfill(4)[::-1]
    digit_positions = [10, 7, 4, 1]
    
    current_display = [set() for _ in range(4)]
    if hasattr(lh, '_current_display'):
        current_display = lh._current_display
    
    for idx, (digit_char, col_start) in enumerate(zip(digits_str, digit_positions)):
        if number < 10 ** idx:
            continue
            
        digit = int(digit_char)
        new_wells = set(get_digit_wells(digit, col_start))
        old_wells = current_display[idx]
        
        wells_to_remove = old_wells - new_wells
        wells_to_add = new_wells - old_wells
        
        for well_id in wells_to_remove:
            tip_spot = await get_next_tip_spot(lh)
            await lh.pick_up_tips(tip_spots=tip_spot, use_channels=[0])
            await lh.aspirate(dino_plate[well_id], vols=[360])  
            await lh.dispense(trough["A1"], vols=[360])  
            await lh.discard_tips()
        
        for well_id in wells_to_add:
            tip_spot = await get_next_tip_spot(lh)
            await lh.pick_up_tips(tip_spots=tip_spot, use_channels=[0])
            await lh.aspirate(trough["A1"], vols=[360])  
            await lh.dispense(dino_plate[well_id], vols=[360])  
            await lh.discard_tips()
        
        current_display[idx] = new_wells
    
    lh._current_display = current_display

In [19]:
async def make_dini_ot2():
    deck = OTDeck()
    deck.assign_child_at_slot(corning_96_wellplate_360ul_flat(name=f"dino_plate"), 5)
    deck.assign_child_at_slot(agilent_1_reservoir_290ml(name="trough"), 6)
    
    tip_rack_slots = [1, 2, 3]
    for i, tip_slot in enumerate(tip_rack_slots):
        deck.assign_child_at_slot(opentrons_96_tiprack_1000ul(name=f"tip_rack_{i}"), tip_slot)
    
    return deck

# call function
deck = await make_dini_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

# set liquid after lh is created
trough = lh.deck.get_resource("trough")
trough.get_item('A1').tracker.set_liquids([(None, 290000)])

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash_container was assigned to the liquid handler.
Resource dino_plate was assigned to the liquid handler.
Resource trough was assigned to the liquid handler.
Resource tip_rack_0 was assigned to the liquid handler.
Resource tip_rack_1 was assigned to the liquid handler.
Resource tip_rack_2 was assigned to the liquid handler.
Websocket server started at ws://127.0.0.1:2124
File server started at http://127.0.0.1:1340


In [20]:
import time
for i in range(11):
    print(f"Displaying: {i}")
    await display_number(lh, i)
    time.sleep(1)

Displaying: 0
Displaying: 1
Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tip_rack_0_A1        0,0,0            Tip          1000.0           7.95                 88               No        
Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 360.0    trough_A1            0,0,0            None       None       None       
Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 360.0    dino_plate_D12       0,0,0            None       None       None       
Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: trash                0,0.0,0          Tip          1000.0           7.95                 88               No        
Picking up tips:
pip#  resource             offset           tip type     